### 1. 데이터 로드 및 확인

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('./netflix_customer_churn.csv')

In [ ]:
display(df.head())
print(f'df.shape : {df.shape}, df.size : {df.size}')

In [ ]:
display(df.info())

In [ ]:
df.favorite_genre.unique()

#### 1.1. 결측치 확인 및 상관 관계 분석

In [ ]:
# 결측치 및 고유값 확인
print("결측치 확인:")
print(df.isna().sum())
print("\n고유값 개수:")
print(df.nunique())

# 상관관계 분석
print("\n=== 수치형 변수 상관관계 분석 ===")
# 수치형 컬럼만 선택
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
print(f"\n수치형 컬럼: {list(numeric_cols)}")

# 상관관계 행렬
correlation_matrix = df[numeric_cols].corr()
print("\n상관관계 행렬:")
print(correlation_matrix)

# 타겟 변수(churned)와의 상관관계 확인
print("\nchurned와의 상관관계:")
print(correlation_matrix['churned'].sort_values(ascending=False))

# 상관관계 히트맵 시각화
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', 
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Correlation Heatmap - Numeric Features', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# churned와 높은 상관관계를 가진 변수 찾기 (절댓값 기준)
churned_corr = correlation_matrix['churned'].drop('churned').abs().sort_values(ascending=False)
print("\nchurned와 상관관계가 높은 변수 (절댓값 기준):")
print(churned_corr.head())

#### 1.2. 상관관계가 높은 watch_hours 이상치 확인

In [ ]:
# 기본 통계량
print("기본 통계량:")
print(df['watch_hours'].describe())

# IQR 방법으로 이상치 탐지
Q1 = df['watch_hours'].quantile(0.25)
Q3 = df['watch_hours'].quantile(0.75)
IQR = Q3 - Q1

# 이상치 경계값 계산
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"\nIQR 기반 이상치 경계:")
print(f"   Q1 (25%): {Q1:.2f}")
print(f"   Q3 (75%): {Q3:.2f}")
print(f"   IQR: {IQR:.2f}")
print(f"   Lower Bound (Q1 - 1.5*IQR): {lower_bound:.2f}")
print(f"   Upper Bound (Q3 + 1.5*IQR): {upper_bound:.2f}")

# Z-Score 방법 (표준편차 기반)
from scipy import stats
z_scores = np.abs(stats.zscore(df['watch_hours']))
z_outliers = df[z_scores > 3]  # Z-score > 3인 경우 이상치

print(f"\n4. Z-Score 기반 이상치 (|Z| > 3): {len(z_outliers)}개 ({len(z_outliers)/len(df)*100:.2f}%)")

# 시각화
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 박스플롯
axes[0, 0].boxplot(df['watch_hours'], vert=False)
axes[0, 0].axvline(lower_bound, color='r', linestyle='--', label='Lower Bound')
axes[0, 0].axvline(upper_bound, color='r', linestyle='--', label='Upper Bound')
axes[0, 0].set_xlabel('Watch Hours')
axes[0, 0].set_title('Boxplot - Watch Hours (with IQR boundaries)')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 히스토그램
axes[0, 1].hist(df['watch_hours'], bins=50, edgecolor='black', alpha=0.7)
axes[0, 1].axvline(lower_bound, color='r', linestyle='--', label='Lower Bound')
axes[0, 1].axvline(upper_bound, color='r', linestyle='--', label='Upper Bound')
axes[0, 1].set_xlabel('Watch Hours')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('Distribution - Watch Hours')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Violin Plot
axes[1, 0].violinplot(df['watch_hours'], vert=False, showmeans=True, showmedians=True)
axes[1, 0].set_xlabel('Watch Hours')
axes[1, 0].set_title('Violin Plot - Watch Hours')
axes[1, 0].grid(True, alpha=0.3)


plt.tight_layout()
plt.show()

#### 1.3. 필요없는 columns 제거

In [ ]:
# customer_id(ID), monthly_fee(월 이용료), payment_method(결제 수단), avg_watch_time_per_day(평균 시청 시간) 컬럼 제거
# 평균 시청 시간의 경우 이상치가 많고 watch_hours와 유사하기 때문에 drop
drop_df = df.drop(columns=['customer_id', 'monthly_fee', 'payment_method', 'avg_watch_time_per_day'])
drop_df.head()

### 2. One-Hot Encoding

#### 2.1. One-Hot이 필요한 컬럼
- **gender(성별)** : 'Female', 'Male', 'Other'로 나뉨
- subscription_type(구독 타입) : 'Basic', 'Standard', 'Premium'으로 나뉨
- region(지역) : 대륙으로 나뉨
- device(시청 기기) : 'TV', 'Mobile', 'Laptop', 'Desktop', 'Tablet'로 나뉨
- favorite_genre(선호하는 장르) : 'Action', 'Sci-Fi', 'Drama', 'Horror', 'Romance', 'Comedy', 'Documentary'로 나뉨

In [ ]:
# gender, subscription_type, region, device, favorite_genre one hot True = 1 / False = 0
onehot_df = pd.get_dummies(drop_df, columns=['gender', 'subscription_type', 'region', 'device', 'favorite_genre'], drop_first=False).astype(int)

display(onehot_df.head()) 
display(onehot_df.columns)

#### 2.2. watch_hours 음수 값 확인

In [ ]:
# watch_hours 결측치 확인 0 포함
display(onehot_df[['watch_hours']].isna().sum())
(onehot_df['watch_hours'].values < 0).sum()

#### 2.3. watch_hours Standard Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
onehot_df['watch_hours_scaled'] = scaler.fit_transform(onehot_df[['watch_hours']])

# 스케일링 전후 비교
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.histplot(onehot_df['watch_hours'], bins=30, kde=True)
plt.title('Before Scaling')
plt.subplot(1, 2, 2)
sns.histplot(onehot_df['watch_hours_scaled'], bins=30, kde=True)
plt.title('After Scaling')
plt.show()

In [ ]:
onehot_df.describe()

#### 2.4. age Preprocessing

##### 2.4.1. age Ohe-Hot

In [ ]:
# age - one hot by age group (drop False)
age_bins = [0, 18, 25, 35, 45, 55, 65, 100]
age_labels = ['0-18', '19-25', '26-35', '36-45', '46-55', '56-65', '65+']
onehot_df['age_group'] = pd.cut(onehot_df['age'], bins=age_bins, labels=age_labels, right=False)
onehot_df = pd.get_dummies(onehot_df, columns=['age_group'], drop_first=False).astype(int)
onehot_df.head()

##### 2.4.2. age Min-Max Scaling

In [ ]:
# age Scaling by MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

# 나이 스케일링
scaler = MinMaxScaler()
onehot_df['age_scaled'] = scaler.fit_transform(onehot_df[['age']])

# 스케일링 전후 비교
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.histplot(onehot_df['age'], bins=30, kde=True)
plt.title('Before Scaling (Age)')
plt.subplot(1, 2, 2)
sns.histplot(onehot_df['age_scaled'], bins=30, kde=True)
plt.title('After Scaling (Age)')
plt.show()

# 원본 데이터와 스케일링된 데이터의 처음 몇 행 확인
print("\n원본 나이와 스케일링된 나이 비교:")
print(onehot_df[['age', 'age_scaled']].head())

#### 2.5. 확인

In [ ]:
onehot_df

In [ ]:
onehot_df.info()

In [ ]:
onehot_df.describe()

#### 2.6. One-Hot Encoidng csv로 저장

In [ ]:
# csv 파일로 저장
onehot_df.to_csv('./netflix_customer_onehot_preprocessed.csv', index=False)
print("\nOne-Hot Encoding 전처리 데이터 저장 완료: './netflix_customer_onehot_preprocessed.csv'")

In [ ]:
onehot_df.columns

### 3. Label Encoding

#### 3.1. 데이터 로드 및 필요 없는 columns drop

In [ ]:
# 원본 데이터 다시 불러오기
tree_df = drop_df.copy()
print(f"컬럼 수: {tree_df.shape[1]}\n")

#### 3.2. Label Encoding

In [ ]:
# 2. 범주형 변수 Label Encoding
from sklearn.preprocessing import LabelEncoder

categorical_cols = ['gender', 'subscription_type', 'region', 'device', 'favorite_genre']
print(f"Label Encoding 적용할 범주형 컬럼: {categorical_cols}")

# Label Encoder 딕셔너리 저장
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    tree_df[f'{col}_encoded'] = le.fit_transform(tree_df[col])
    label_encoders[col] = le
    print(f"   - {col}: {tree_df[col].unique()} -> {tree_df[f'{col}_encoded'].unique()}")

# 원본 범주형 컬럼 제거
tree_df = tree_df.drop(columns=categorical_cols)
print(f"\n인코딩 후 컬럼 수: {tree_df.shape[1]}\n")

In [ ]:
# 4. 최종 데이터 확인
print("최종 Tree 모델용 데이터:")
display(tree_df.head(10))
print(f"\nShape: {tree_df.shape}")
print(f"\n컬럼 목록:")
print(tree_df.columns.tolist())

In [ ]:
# 5. 데이터 타입 확인
print("\n데이터 타입:")
print(tree_df.dtypes)

# 6. CSV 파일로 저장
tree_df.to_csv('./netflix_customer_churn_tree_preprocessed.csv', index=False)
print("\nTree 모델용 전처리 데이터 저장 완료: './netflix_customer_churn_tree_preprocessed.csv'")